In [1]:
#DISCLAIMER it's messy code, just ignore this file

from graph_tool.all import *
from numpy.random import randint
from numpy.random import random
from numpy.random import *
from IPython.display import Image #import to display images
from matplotlib import *
from gi.repository import Gtk, Gdk, GdkPixbuf, GObject, GLib
import sys, os, os.path
import time
import numpy as np
import matplotlib.pyplot as plt
#things to be done, figure out all the movement and movement in between states
#add different infection probabilities for vaccination and age group
#if person is vaccinated and not vaccinated


In [2]:
#PRE:  g is graph, distribution is string with country name. distribution must be for 20 age groups, normalized to
#      number of vertices in graph g
#POST: assigns age property randomly to the vertices in g, according to distribution

def add_ages(g,distribution):
    path1 = 'Age_Dists/'
    path2 = 'Dist.csv'
    #generate array containing amount of people per age group
    agedistribution = np.genfromtxt(path1+distribution+path2, delimiter=',')[:,1]
    #visit vertices in random order
    print(agedistribution)
    vs = list(g.vertices())
    shuffle(vs)
    #we will fill all age groups one by one
    a = 1 #current agegroup
    counter = 0 #counter for how many people have been placed in current group
    
    for v in vs:
        if(a > 20):
            #safety measure
            return
        if counter < agedistribution[a-1]:
            #if current age group hasn't gotten enough people assigned, add another, move on to next vertex
            g.vp.age[v] = a
            agegrouplists[a-1].append(v)
            counter = counter + 1
        else:
            #current age group is full, move to next age group with at least one person
            a = a + 1
            while(agedistribution[a-1] == 0):
                if(a < 20):
                    a = a + 1
                else:
                    #in case number for last age group is 0
                    return
            g.vp.age[v] = a
            agegrouplists[a-1].append(v)
            counter = 1
        

In [3]:
def update_state():
    #these global varables are meant to save the number of people of each g.vp.state in the correponding list
    global sval
    global ival
    global rval
    global vval
    global dval
    global scount
    global icount
    global vcount
    global rcount
    global dcount
    global economy_list
    global tlist
    
    global tags
    global tag
    global TIMEUNIT
    g.vp.newly_infected.a = False
    
    economy_value = 0
    g.vp.removed.a = False
    #global variable ccount tracks current timestamp for the graph, for now it's set that the graph will be drawn after 20 days (time units)
    global ccount
    #draw a graph
    if ccount==TIMEUNIT:
        plt.plot(tlist, sval, label = "S")
        plt.plot(tlist, rval, label = "R")
        plt.plot(tlist, vval, label = "V")
        plt.plot(tlist, ival, label = "I")
        plt.plot(tlist, dval, label = "D")
        plt.plot(tlist,economy_list, label = "Economy")
        plt.legend()
        plt.show()
     
    #increment timestamp
    ccount+=1  
    # visit the nodes in random order
    

    vs = list(g.vertices())

    shuffle(vs)

    for v in vs:

        if g.vp.state[v] == I: #infected
            
            if g.vp.vac[v]==False and random() < reclist[g.vp.age[v]-1] : #r: recovery rate
                rcount+=1
                icount-=1
                tag = 'a' #don't mind these tags, i want them to help me debugging (cd)
                g.vp.state[v] = R
                
            elif g.vp.vac[v] == True and random() < vac_reclist[g.vp.age[v]-1]:
                rcount+=1
                icount-=1
                tag = 'b'
                g.vp.state[v] = R
                
            elif g.vp.vac[v] == False and random() < drlist[g.vp.age[v]-1]:
                g.vp.state[v] = D       #dead
                dcount+=1
                icount-=1
                tag = 'c'
                g.clear_vertex(v)  # red circle with no adjacent edges
                
            elif g.vp.vac[v] == True and random() < vac_drlist[g.vp.age[v]-1]:
            
                g.vp.state[v] = D       #g.vp.vac dead
                dcount+=1
                icount-=1
                tag = 'd'
                g.clear_vertex(v)  # red circle with no adjacent edges

        elif g.vp.state[v] == S:
            economy_value += economy[g.vp.age[v]-1]
            if random() < vacc_av[g.vp.age[v]-1]*vacc_ac[g.vp.age[v]-1]:
                
                g.vp.state[v] = V #if g.vp.vaccine is available and accepted by individual, g.vp.vaccinate
                g.vp.vac[v] = True
                vcount+=1
                scount-=1
            
            else: 
                ns = list(v.out_neighbors())
                
                i = 0

                for w in ns: #iterate through all neighbors, each has infection probability

                    if g.vp.state[w] == I and random()<inflist[g.vp.age[v]-1]:

                        #infection rate dependent on g.vp.age
                        i = 1 
                if i:#if infected by at least one neighbor, change status
                    g.vp.state[v] = I
                    icount+=1
                    tag = 'z'
                    scount-=1
                    g.vp.newly_infected[v] = True
    
        elif g.vp.state[v] == V:
            economy_value += economy[g.vp.age[v]-1]
            ns = list(v.out_neighbors())

            if len(ns) > 0:

                w = ns[randint(0, len(ns))]  # choose a random neighbor

                if g.vp.state[w] == I:

                    #infection rate dependent on g.vp.age
                    if random()<vac_inflist[g.vp.age[v]-1]:
                            icount+=1
                            tag = 'zz'
                            vcount-=1
                            g.vp.state[v] = I

                            g.vp.newly_infected[v] = True
                        

        if g.vp.state[v] == R:
            economy_value += economy[g.vp.age[v]-1]
            g.vp.removed[v] = True
    if icount<0:
        raise Exception("Error, icount can't be less than zero")
    #print (icount)
    ival.append(icount)
    vval.append(vcount)
    sval.append(scount)
    rval.append(rcount)
    dval.append(dcount)
    tags.append(str(ccount)+tag)
    economy_list.append(economy_value)

    # Filter out the recovered vertices

    g.set_vertex_filter(g.vp.removed, inverted=True)


    # The following will force the re-drawing of the graph, and issue a

    # re-drawing of the GTK window.

    win.graph.regenerate_surface()
    time.sleep(0.2)
    win.graph.queue_draw()


    # if doing an offscreen animation, dump frame to disk

    if offscreen:

        global count

        pixbuf = win.get_pixbuf()

        pixbuf.savev(r'./frames/sirs%06d.png' % count, 'png', [], [])

        if count > max_count:

            sys.exit(0)

        count += 1


    # We need to return True so that the main loop will call this function more

    # than once.

    return True

In [4]:
# SET UP GRAPH
def make_graph():
    size = 100
    
    g = Graph(directed=False)
    
    #definition of vertex properties
    global S          # White color
   
    global I        # Black color
    
    global R     # Grey color (will not actually be drawn)
     
    global V        # Blue color
    
    global D        #DEATH (red)
    
    #timestamp list, basically [1,2,...]
    global tlist 
    #TIMEUNIT defines how many time units it should take until graph plotting
    global TIMEUNIT
    for i in range(TIMEUNIT):
        tlist.append(i+1)
    
    #lists to track the number of people of each state
    global sval
    global ival
    global rval
    global vval
    global dval
    global economy_list
    global scount
    global icount
    global vcount
    global rcount
    global dcount
    global ccount
    
    global tags
    #initialize tag as empty string
    tag =""
    #all definitions of vertice edges
    state = g.new_vertex_property("vector<double>")
    age=g.new_vertex_property("int")
    vac=g.new_vertex_property("bool")
    removed = g.new_vertex_property("bool")
    newly_infected = g.new_vertex_property("bool")
    
    g.vp.state = state
    g.vp.age = age
    g.vp.vac = vac
    g.vp.removed = removed 
    g.vp.newly_infected = newly_infected
    global agegroups
    global agegrouplists 

    # insert random vertices (nodes)
    g.add_vertex(size)
    
    # insert some random links
    for s,t in zip(randint(0, size, 2*size), randint(0, size, 2*size)):
        g.add_edge(g.vertex(s), g.vertex(t))
    
    global scount
    global vcount
    #makes all nodes suceptible
    for i in g.vertices():
        v = g.vertex(i) 
        
        g.vp.state[v] = S
        scount+=1
        vac[v]=False;
        
        '''if random() < vac_prob:
        
            vac[v]=True;
            vcount+=1
            scount-=1
            state[v] = V'''
    
    #this line is to randomlyinitialize one vertex as infected
    state[randint(0,size)]= I
    icount += 1
    #assign ages to vertices following given age distribution
    add_ages(g,'Japan')
    return g
    #print(graph_tool.spectral.adjacency(g,weight=None, vindex=None, operator=True)*np.identity(99))

In [5]:
#definition of vertex properties
S = [1, 1, 1, 1]           # White color

I = [0, 0, 0, 1]           # Black color

R = [0.5, 0.5, 0.5, 1.]    # Grey color (will not actually be drawn)

V = [0, 0, 1, 1]           # Blue color

D = [0.8, 0, 0, 0.6]       #DEATH (red)
agegroups=20
agegrouplists = [[] for i in range(1, agegroups+1)]
tlist = list()
TIMEUNIT = 20
tags = list()
#probabilities


#death probability list for different age groups: https://pubmed.ncbi.nlm.nih.gov/32674819/
drlist = [.003,.003,.003,.003,.003,.003,.005,.005,.011,.011,.03,.03,.095,.095,.228,.228,.296,.296,.296,0.296]
#same VACCINATED
vac_drlist = [.001,.001,.001,.001,.001,.001,.002,.002,.005,0.005,0.015,0.015,.047,.047,.114,.114,.148,.148,.148,.148]


#vaccine availability by age group:
vacc_av = [0,0,0,0,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01]
#vaccine acceptance rate by age group:
vacc_ac = [0,0,0,0,0.5,0.5,0.5,0.5,0.6,0.6,0.6,0.6,0.7,0.7,0.7,0.7,0.8,0.8,0.9,0.9]

#infection probability for different age groups
inflist = [0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8]
#same VACCINATED
vac_inflist = [.20,.20,.20,.20,.20,.20,.20,.20,.20,.20,.20,.20,.20,.20,.20,.20,.20,.20,.20,0.20]

#recovery probability for different age groups
reclist = [0.34,0.34,0.34,0.34,0.34,0.34,0.34,0.34,0.32,0.32,0.28,0.28,0.22,0.22,0.15,0.15,0.10,0.10,0.08,0.08]
#same VACCINATED
vac_reclist = [0.39,0.39,0.39,0.39,0.39,0.39,0.39,0.39,0.35,0.35,0.31,0.31,0.26,0.26,0.20,0.20,0.14,0.14,0.13,0.13]

#economy contribution

economy = [0,0,0,0.025,0.025,0.05,0.1,0.1,0.1,0.2,0.1,0.1,0.1,0.05,0.025,0.025,0,0,0]
#lists to track the number of people of each state
sval=list()
ival=list()
rval=list()
vval=list()
dval=list()
economy_list=list()
scount=0
icount=0
vcount=0
rcount=0
dcount=0
ccount=0
g = make_graph()
# layout positions
pos = sfdp_layout(g)
def graph_get():
    return g
#Displaying how the graph will be initialized
graph_draw(g, vertex_text=g.vertex_index, output="two-nodes.png")
fig = Image(filename=('two-nodes.png'))
fig


# If True, the frames will be dumped to disk as images.
# CAN SOMONE FIGURE OUT WHAT THIS DOES???????????????????????????????
#if the argument passed from command line=="offscreen" then do the thing else false

offscreen = sys.argv[1] == "offscreen" if len(sys.argv) > 1 else False

max_count = 500

if offscreen and not os.path.exists("./frames"):

    os.mkdir("./frames")
#?????????????????????????????????????????????????????????????????????


# This creates a GTK+ window with the initial graph layout,basically creates the pop up window

if not offscreen:

    win = GraphWindow(g, pos, geometry=(500, 400),
                      

                      edge_color=[0.6, 0.6, 0.6, 1],

                      vertex_fill_color=g.vp.state,

                      vertex_halo=g.vp.newly_infected,

                      vertex_halo_color=[0.8, 0, 0, 0.6],)

else:

    count = 0

    win = Gtk.OffscreenWindow()

    win.set_default_size(500, 400)

    win.graph = GraphWidget(g, pos,
                            
                            edge_color=[0.6, 0.6, 0.6, 1],

                            vertex_fill_color=state,
                            
                            vertex_halo=newly_infected,

                            vertex_halo_color=[0.8, 0, 0, 0.6])

    win.add(win.graph)

[4. 4. 4. 4. 5. 5. 5. 6. 7. 8. 7. 6. 6. 6. 7. 6. 4. 3. 2. 0.]


In [6]:
for i in range(1,agegroups+1):
    print(i,': ',len(agegrouplists[i-1]))

1 :  4
2 :  4
3 :  4
4 :  4
5 :  5
6 :  5
7 :  5
8 :  6
9 :  7
10 :  8
11 :  7
12 :  6
13 :  6
14 :  6
15 :  7
16 :  6
17 :  4
18 :  3
19 :  2
20 :  0


In [7]:
#drlist[age[10]]

In [8]:
cid = GLib.idle_add(update_state)


# We will give the user the ability to stop the program by closing the window.

win.connect("delete_event", Gtk.main_quit)


# Actually show the window, and start the main loop.

win.show_all()

Gtk.main()

In [9]:
print(tags)
print("economy=", economy_list)

['1a', '2a', '3a', '4a', '5a', '6a', '7a', '8a', '9a', '10a', '11a', '12a', '13a', '14a', '15a', '16a', '17a', '18a', '19a']
economy= [6.399999999999998, 6.399999999999999, 6.3999999999999995, 6.399999999999999, 6.399999999999995, 6.399999999999999, 6.3999999999999995, 6.399999999999995, 6.3999999999999995, 6.399999999999998, 6.399999999999999, 6.399999999999998, 6.399999999999996, 6.399999999999999, 6.399999999999997, 6.399999999999999, 6.399999999999997, 6.399999999999998, 6.399999999999998]


In [10]:
g.list_properties()
graph_get()

age                (vertex)  (type: int32_t)
newly_infected     (vertex)  (type: bool)
removed            (vertex)  (type: bool)
state              (vertex)  (type: vector<double>)
vac                (vertex)  (type: bool)


<Graph object, undirected, with 99 vertices and 197 edges, 5 internal vertex properties, edges filtered by (<EdgePropertyMap object with value type 'bool', for Graph 0x7f60af3e4310, at 0x7f60af3f3b80>, False), vertices filtered by (<VertexPropertyMap object with value type 'bool', for Graph 0x7f60af3e4310, at 0x7f60af3f3e50>, True), at 0x7f60af3e4310>

In [11]:
g.save("my_graph.xml.gz")
g2 = load_graph("my_graph.xml.gz")